In [1]:
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
!pip install snowballstemmer
!pip install gensim

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jananisundaresan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim
import re

In [3]:
#!pip install spacy
import spacy
spacy.load('en')
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [4]:
from nltk.tokenize import sent_tokenize,word_tokenize

In [5]:
data=pd.read_csv("~/Desktop/trun_1900_2019.csv")

In [20]:
data=data[(data['publication_year']>2000.0)&(data['publication_year']<=2001.0)]
data.shape

(24749, 20)

In [21]:
#shuffle data
data = data.sample(frac=1)

In [22]:
data['len']=([len(w.split(" ")) for w in data['description']])

In [23]:
datanew=data[(data['len']>25)&(data['len']<=100)]

In [24]:
datanew.shape

(10079, 21)

In [25]:
datanew=datanew.iloc[:5000,:]

In [26]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text


In [27]:
%time data['description']=data['description'].map(lambda x:clean_text(x))

CPU times: user 38.4 s, sys: 984 ms, total: 39.4 s
Wall time: 39.7 s


In [28]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  


In [29]:
# Convert to list
data = data['description'].values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['justic', 'lewi', 'powel', 'jr', 'absorb', 'readabl', 'biographi', 'one', 'import', 'suprem', 'court', 'justic', 'sinc', 'world', 'war', 'ii']]


In [30]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  # processed Text Data!

In [31]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
%time
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=10,
                                           per_word_topics=True)

pprint(lda_model.print_topics())

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs
[(0,
  '0.024*"book" + 0.011*"write" + 0.011*"includ" + 0.011*"work" + 0.008*"new" '
  '+ 0.007*"reader" + 0.007*"author" + 0.006*"cultur" + 0.006*"show" + '
  '0.005*"well"'),
 (1,
  '0.000*"naa" + 0.000*"kinaa" + 0.000*"theke" + 0.000*"smyy" + 0.000*"kre" + '
  '0.000*"upr" + 0.000*"pul" + 0.000*"chil" + 0.000*"kaahiniir" + '
  '0.000*"ghttnaaprmpraa"'),
 (2,
  '0.039*"child" + 0.028*"die" + 0.020*"meet" + 0.019*"parent" + '
  '0.016*"husband" + 0.015*"centuri" + 0.015*"und" + 0.014*"van" + 0.012*"een" '
  '+ 0.012*"den"'),
 (3,
  '0.019*"que" + 0.016*"secret" + 0.016*"murder" + 0.011*"son" + '
  '0.010*"return" + 0.009*"kill" + 0.009*"dark" + 0.008*"dan" + 0.007*"danger" '
  '+ 0.007*"killer"'),
 (4,
  '0.027*"food" + 0.026*"anim" + 0.025*"simpl" + 0.018*"recip" + '
  '0.017*"garden" + 0.016*"eat" + 0.013*"kitchen" + 0.013*"kingdom" + '
  '0.012*"perhap" + 0.012*"bright"'),
 (5,
  '0.000*"naa" + 0.000*"kinaa" + 0.000*"t

In [32]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,9.0,0.7010,"life, stori, world, find, time, love, live, ma...","[justic, lewi, powel, jr, absorb, readabl, bio..."
1,1,9.0,0.6332,"life, stori, world, find, time, love, live, ma...","[pope, breakfast, madelin, bean, frittata, ski..."
2,2,9.0,0.5310,"life, stori, world, find, time, love, live, ma...","[paperback, enorm, popular, essay, fanni, flag..."
3,3,0.0,0.7403,"book, write, includ, work, new, reader, author...","[book, give, introduct, probabl, mani, practic..."
4,4,0.0,0.9400,"book, write, includ, work, new, reader, author...","[seri, standard, social, studi, cultur, indivi..."
5,5,9.0,0.6564,"life, stori, world, find, time, love, live, ma...","[latest_instal, ralph, mcinerni, belov, seri, ..."
6,6,9.0,0.5242,"life, stori, world, find, time, love, live, ma...","[american, middl, obe, epidem, contrast, frenc..."
7,7,9.0,0.7125,"life, stori, world, find, time, love, live, ma...","[someth, stop, nita, hand, bookshelf, look, fi..."
8,8,9.0,0.8947,"life, stori, world, find, time, love, live, ma...","[david, macfarlan, summer, goneintroduc, write..."
9,9,0.0,0.5374,"book, write, includ, work, new, reader, author...","[new, american, ident, forg, immigr, expan, ba..."


In [33]:
import seaborn as sns
import matplotlib.colors as mcolors

In [34]:
# Get topic weights and dominant topics ------------
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook

# Get topic weights
topic_weights = []
for i, row_list in enumerate(lda_model[corpus]):
    topic_weights.append([w for i, w in row_list[0]])

# Array of topic weights    
arr = pd.DataFrame(topic_weights).fillna(0).values

# Keep the well separated points (optional)
arr = arr[np.amax(arr, axis=1) > 0.35]

# Dominant topic number in each doc
topic_num = np.argmax(arr, axis=1)

# tSNE Dimension Reduction
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
tsne_lda = tsne_model.fit_transform(arr)

# Plot the Topic Clusters using Bokeh
output_notebook()
n_topics = 4
mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), 
              plot_width=900, plot_height=700)
plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])
show(plot)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 24566 samples in 0.014s...
[t-SNE] Computed neighbors for 24566 samples in 0.861s...
[t-SNE] Computed conditional probabilities for sample 1000 / 24566
[t-SNE] Computed conditional probabilities for sample 2000 / 24566
[t-SNE] Computed conditional probabilities for sample 3000 / 24566
[t-SNE] Computed conditional probabilities for sample 4000 / 24566
[t-SNE] Computed conditional probabilities for sample 5000 / 24566
[t-SNE] Computed conditional probabilities for sample 6000 / 24566
[t-SNE] Computed conditional probabilities for sample 7000 / 24566
[t-SNE] Computed conditional probabilities for sample 8000 / 24566
[t-SNE] Computed conditional probabilities for sample 9000 / 24566
[t-SNE] Computed conditional probabilities for sample 10000 / 24566
[t-SNE] Computed conditional probabilities for sample 11000 / 24566
[t-SNE] Computed conditional probabilities for sample 12000 / 24566
[t-SNE] Computed conditional probabilities for sam

Loading BokehJS ...

In [35]:
df_dominant_topic['Dominant_Topic'].value_counts()

9.0    15569
0.0     4938
8.0     2275
3.0     1247
2.0      646
1.0       30
4.0       24
5.0       11
7.0        7
6.0        2
Name: Dominant_Topic, dtype: int64

In [36]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.386394  0.129882       1        1  46.583954
0     -0.353724 -0.000385       2        1  25.686428
3     -0.051798 -0.395693       3        1  11.426215
8      0.031777  0.030787       4        1   9.690739
2      0.062553  0.055712       5        1   4.571207
4      0.140832  0.047538       6        1   1.146367
6      0.139188  0.033040       7        1   0.223772
7      0.139188  0.033040       8        1   0.223772
5      0.139188  0.033040       9        1   0.223772
1      0.139188  0.033040      10        1   0.223772, topic_info=       Category          Freq         Term         Total  loglift  logprob
17      Default  10083.000000         book  10083.000000  30.0000  30.0000
757     Default   2940.000000        child   2940.000000  29.0000  29.0000
2170    Default   3540.000000          que   3540.000000  28.0000  28.0000
55      Default   5513.000000          man   5513.000000  27.0000  27.0000
51      Default   8893.000000         life   8893.000000  26.0000  26.0000
825     Default   2129.000000          die   2129.000000  25.0000  25.0000
1039    Default   3016.000000       secret   3016.000000  24.0000  24.0000
209     Default   3013.000000       murder   3013.000000  23.0000  23.0000
709     Default   4586.000000       includ   4586.000000  22.0000  22.0000
748     Default   4979.000000        write   4979.000000  21.0000  21.0000
420     Default   7629.000000        stori   7629.000000  20.0000  20.0000
271     Default   5510.000000         work   5510.000000  19.0000  19.0000
330     Default   2825.000000          son   2825.000000  18.0000  18.0000
36      Default   6060.000000         find   6060.000000  17.0000  17.0000
1121    Default   1463.000000         meet   1463.000000  16.0000  16.0000
802     Default   1430.000000       parent   1430.000000  15.0000  15.0000
1675    Default   1795.000000          che   1795.000000  14.0000  14.0000
2337    Default   1908.000000       return   1908.000000  13.0000  13.0000
52      Default   5379.000000         love   5379.000000  12.0000  12.0000
247     Default   5164.000000         live   5164.000000  11.0000  11.0000
792     Default   1214.000000      husband   1214.000000  10.0000  10.0000
1731    Default   1589.000000          una   1589.000000   9.0000   9.0000
220     Default   4862.000000        woman   4862.000000   8.0000   8.0000
1766    Default   1644.000000         kill   1644.000000   7.0000   7.0000
756     Default   1148.000000      centuri   1148.000000   6.0000   6.0000
769     Default   1618.000000         dark   1618.000000   5.0000   5.0000
4088    Default   1495.000000          del   1495.000000   4.0000   4.0000
149     Default   6684.000000          new   6684.000000   3.0000   3.0000
2531    Default   1098.000000          und   1098.000000   2.0000   2.0000
666     Default   1081.000000          van   1081.000000   1.0000   1.0000
...         ...           ...          ...           ...      ...      ...
151528  Topic10      0.024157     prtibaad      0.823453   2.5734 -11.9290
151537  Topic10      0.024157      tthunko      0.823453   2.5734 -11.9290
151541  Topic10      0.024157       yubker      0.823453   2.5734 -11.9290
72119   Topic10      0.024157        kintu      0.823453   2.5734 -11.9290
151536  Topic10      0.024157   ttaalignyj      0.823453   2.5734 -11.9290
151529  Topic10      0.024157         resh      0.823453   2.5734 -11.9290
151535  Topic10      0.024157      thaanaa      0.823453   2.5734 -11.9290
4211    Topic10      0.024157         dite      0.823453   2.5734 -11.9290
151532  Topic10      0.024157      sbismyy      0.823453   2.5734 -11.9290
151503  Topic10      0.024157         caap      0.823453   2.5734 -11.9290
151501  Topic10      0.024157      bishess      0.823453   2.5734 -11.9290
151499  Topic10      0.024157     baancaat      0.823453   2.5734 -11.9290
151498  T

In [ ]:
lda_model.show_topics(num_words=50,num_topics=5)